In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum

In [33]:
fact_internet_sales_df = spark.read.csv("fact_internet_sales.csv", header=True, inferSchema=True)
dim_product_df = spark.read.csv("dim_product.csv", header=True, inferSchema=True)
dim_customer_df = spark.read.csv("dim_customer.csv", header=True, inferSchema=True)
dim_date_df = spark.read.csv("dim_date.csv", header=True, inferSchema=True)
dim_currency_df = spark.read.csv("dim_currency.csv", header=True, inferSchema=True)

In [34]:
# dim_date_df = dim_date_df.filter(col('EnglishDayNameOfWeek')=='Sunday')

In [41]:
# dim_product_df = dim_product_df.filter((col('Color') == 'Silver') & (col('Size').isNotNull()) & (col('Weight')>10))

In [44]:
# dim_customer_df = dim_customer_df.filter((col('YearlyIncome') >100_000) & (col('TotalChildren') == 1))

In [48]:
(dim_product_df.filter((col('Color') == 'Silver') & (col('Size').isNotNull()) & (col('Weight')>10))
        .join(fact_internet_sales_df, on="ProductKey", how="inner")
        .join(dim_customer_df.filter((col('YearlyIncome') >100_000) & (col('TotalChildren') == 1)), on="CustomerKey", how="inner")
        .join(dim_date_df.filter(col('EnglishDayNameOfWeek')=='Sunday'), fact_internet_sales_df.OrderDateKey == dim_date_df.filter(col('EnglishDayNameOfWeek')=='Sunday').DateKey, how="inner")
).groupBy("CustomerKey", "FirstName").agg(
        sum("TaxAmt").alias("Total Tax Amount"),
        avg("SalesAmount").alias("Average Sales Amount"),
        avg("TotalProductCost").alias("Average Total Product Cost") 
).orderBy('FirstName', ascending = True).show()

+-----------+---------+----------------+--------------------+--------------------------+
|CustomerKey|FirstName|Total Tax Amount|Average Sales Amount|Average Total Product Cost|
+-----------+---------+----------------+--------------------+--------------------------+
|      19696|     Adam|         45.1992|              564.99|                  308.2179|
|      14433| Jennifer|        185.5992|             2319.99|                 1265.6195|
|      15587|  Jessica|        165.7136|           2071.4196|                 1117.8559|
|      11290|  Katelyn|        185.5992|             2319.99|                 1265.6195|
|      15586|   Victor|        185.5992|             2319.99|                 1265.6195|
|      14673|   Willie|        165.7136|           2071.4196|                 1117.8559|
+-----------+---------+----------------+--------------------+--------------------------+

